# Modelos ViT pré-treinados

Ao contrário de CNNs, modelos ViT são extremamente difíceis de treinar "do zero" (a partir de pesos aleatórios). Por isso, é comum que esses modelos sejam inicializados a partir de pesos pré-treinados disponíveis. Uma hipótese atual, ainda sendo estudada, é que modelos pré-treinados em grandes quantidades de dados tendem a possuir melhor acurácia em tarefas *downstream* (classificação, segmentação, etc) do que modelos treinados em menos dados. 

Um paradigma atual de pré-treinamento de modelos é realizar o treinamento em dados **não rotulados**. Isso permite realizar o treinamento em uma quantidade massiva de dados.

Este notebook mostra as funcionalidades da biblioteca timm para carregar modelos pré-treinados.

### Biblioteca Pytorch Image Models (timm)

In [1]:
import torch
import timm

# Modelo ViT utilizando patches de tamanho 16x16 e imagens de entrada de
# tamanho 224x224
tag = 'vit_base_patch16_224'

model = timm.create_model(tag, pretrained=True)
model.eval()

x = torch.rand(1, 3, 224, 224)
with torch.no_grad():
    scores = model(x)

# Scores associados a 1000 classes do ImageNet
scores.shape

torch.Size([1, 1000])

É possível cerregarmos o modelo como um extrator de atributos. Esses atributos podem ser utilizados em outras tarefas como detecção de objetos, segmentação, etc:

In [2]:
model = timm.create_model(tag, pretrained=True, num_classes=0, global_pool='')
model.eval()

with torch.no_grad():
    scores = model(x)

# Token para classificação
cls_token = scores[:,0]
# Mapa de atributos de tamanho 14x14
feature_map = scores[:,1:].reshape(1,14,14,-1)
feature_map.shape

torch.Size([1, 14, 14, 768])

### Modelos treinados no ImageNet

Exemplos de modelos treinados no ImageNet. A técnica de treinamento é descrita no artigo "How to train your ViT? Data, Augmentation, and Regularization in Vision Transformers" https://arxiv.org/abs/2106.10270

* ImageNet 1k: 1281167 imagens separadas em 1000 classes
* ImageNet 21k: 14197122 images separadas em 21841 classes

In [5]:
# Modelo treinado no ImageNet 1k com técnicas robustas de aumento de dados (aug)
# e regularização (reg)
tag = 'vit_base_patch16_224.augreg_in1k'
# Modelo treinado no ImageNet 21k com augreg e refinado no ImageNet 1k
tag = 'vit_base_patch16_224.augreg_in21k_ft_in1k'

### Modelo treinado para recuperar regiões apagadas de imagens

Uma técnica atual muito popular para realizar o pré-treinamento de modelos, chamada de *masked autoencoder* (MAE) consiste em apagar regiões de imagens e treinar um modelo para recuperar essas regiões. O processo é similar ao utilizado no modelo BERT em NLP. A técnica foi definida no artigo "Masked Autoencoders Are Scalable Vision Learners" https://arxiv.org/abs/2111.06377

![](../data/notebook_images/mae.png)

In [6]:
# Treinado com a técnica MAE no ImageNet 1k
tag = 'vit_base_patch16_224.mae'

### Modelo treinado para medir a similaridade entre imagens transformadas

Outra técnica muito popular consiste em treinar um modelo que aprende a similaridade entre imagens. Dado um batch de N imagens, cada imagem sofre duas operações de *data augmentation*. O modelo é treinado para que as saídas associadas à mesma imagem aumentada sejam mais iguais entre si do que a saída para outras imagens do dataset. Para isso, é definida uma função de loss apropriada.

A técnica foi originalmente definida no artigo "A Simple Framework for Contrastive Learning of Visual Representations" (https://arxiv.org/abs/2002.05709). Um dos modelos mais usados atualmente é o DINO ("Emerging Properties in Self-Supervised Vision Transformers" https://arxiv.org/pdf/2104.14294).

![](../data/notebook_images/SimCLR.png)

In [7]:
# Modelo treinado para medir a similaridade entre imagens em um dataset de 142 
# milhões de imagens
tag = 'vit_base_patch14_reg4_dinov2.lvd142m'

### Modelos treinados para medir a similaridade entre textos e imagens

Provavelmente a técnica mais popular atualmente é pré-treinar o modelo em pares de textos e imagens. O modelo aprende a medir a similaridade entre textos e imagens extraídos da web (Flickr, Wikipedia, ou qualquer outro site que possua textos e imagens). O modelo foi originalmente criado pela OpenAI "Learning Transferable Visual Models From Natural Language Supervision" https://arxiv.org/abs/2103.00020. O problema do modelo original é que o dataset utilizado para o treinamento não é conhecido. Por causa disso, foi criado o modelo OpenCLIP.

* OpenAI: 400 milhões de pares de textos e imagens
* OpenCLIP: 2.32 **bilhões** de pares de textos e imagens

In [8]:
# Modelo treinado pela OpenAI, refinado no ImageNet 12k (um dataset específico da
# biblioteca timm baseado no ImageNet) e refinado no ImageNet 1k
tag = 'vit_base_patch16_clip_224.openai_ft_in12k_in1k'
# Modelo OpenCLIP
tag = 'vit_base_patch16_clip_224.laion2b_ft_in12k_in1k'


### Modelo para extração de atributos em múltiplas escalas

O modelo Swin Transformer possui algumas alterações em relação ao ViT baseadas em algumas características de CNNs (em particular, da ResNet). Ele permite extrair atributos de imagens em múltiplas escalas com alta resolução. Isso é possível porque a camada de atenção processa os tokens em janelas ao invés de usar uma atenção global. Modelo definido em "Swin Transformer: Hierarchical Vision Transformer using Shifted Windows" (https://arxiv.org/abs/2103.14030)

![](../data/notebook_images/swin.png)

In [12]:
# Patches de tamanho 4x4. Atenção entre tokens é realizada em janelas de
# tamanho 7x7 tokens
tag = 'swin_base_patch4_window7_224'
model = timm.create_model(tag, pretrained=True, img_size=512)
model.eval()

x = torch.rand(1, 3, 512, 512)
with torch.no_grad():
    scores, features = model.forward_intermediates(x)

# Atributos extraídos pelo modelo de forma similar ao que implementamos usando
# a ResNet
for feat in features:
    print(feat.shape)

torch.Size([1, 128, 128, 128])
torch.Size([1, 256, 64, 64])
torch.Size([1, 512, 32, 32])
torch.Size([1, 1024, 16, 16])
